<a href="https://colab.research.google.com/github/EllieMwangi/NaiveBayes-KNN-Classification/blob/main/Naive_Bayes_Spam_Email_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam Email Detector

### Specifying Analysis Question


Build a naive bayes classifier that detects a spam email

### Defining the Metrics of Success

The model should be able to classify emails as spam or not with at least 80% accuracy.

### Understanding the context

The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...

The collection of spam e-mails came from the creators' postmaster and individuals who had filed spam. The collection of non-spam e-mails came from filed work and personal e-mails, and hence the word 'george' and the area code '650' are indicators of non-spam. These are useful when constructing a personalized spam filter. One would either have to blind such non-spam indicators or get a very wide collection of non-spam to generate a general purpose spam filter.


### Recording the Experimental Design





- Load Data
- Data Cleaning
- Exploratory Data Analysis
- Data Modelling
- Model Evaluation
- Model improvement and tuning
- Conclusion
- Challenging the solution

## Data Loading

In [15]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# Set global parameters
%matplotlib inline
sns.set()
plt.rcParams['figure.figsize'] = (10.0, 8.0)
warnings.filterwarnings('ignore')

In [2]:
with open('spambase.names') as file:
  names = file.read()
  print(names)

| SPAM E-MAIL DATABASE ATTRIBUTES (in .names format)
|
| 48 continuous real [0,100] attributes of type word_freq_WORD 
| = percentage of words in the e-mail that match WORD,
| i.e. 100 * (number of times the WORD appears in the e-mail) / 
| total number of words in e-mail.  A "word" in this case is any 
| string of alphanumeric characters bounded by non-alphanumeric 
| characters or end-of-string.
|
| 6 continuous real [0,100] attributes of type char_freq_CHAR
| = percentage of characters in the e-mail that match CHAR,
| i.e. 100 * (number of CHAR occurences) / total characters in e-mail
|
| 1 continuous real [1,...] attribute of type capital_run_length_average
| = average length of uninterrupted sequences of capital letters
|
| 1 continuous integer [1,...] attribute of type capital_run_length_longest
| = length of longest uninterrupted sequence of capital letters
|
| 1 continuous integer [1,...] attribute of type capital_run_length_total
| = sum of length of uninterrupted sequences of

In [3]:
columns_data= ['word_freq_make',
          'word_freq_address',      
          'word_freq_all',          
          'word_freq_3d',          
          'word_freq_our',          
          'word_freq_over',         
          'word_freq_remove',       
          'word_freq_internet',     
          'word_freq_order',        
          'word_freq_mail',         
          'word_freq_receive',      
          'word_freq_will',         
          'word_freq_people',       
          'word_freq_report',       
          'word_freq_addresses',    
          'word_freq_free',         
          'word_freq_business',     
          'word_freq_email',        
          'word_freq_you',          
          'word_freq_credit',       
          'word_freq_your',         
          'word_freq_font',         
          'word_freq_000',          
          'word_freq_money',        
          'word_freq_hp',           
          'word_freq_hpl',          
          'word_freq_george',       
          'word_freq_650',          
          'word_freq_lab',          
          'word_freq_labs',         
          'word_freq_telnet',       
          'word_freq_857',          
          'word_freq_data',         
          'word_freq_415',          
          'word_freq_85',           
          'word_freq_technology',   
          'word_freq_1999',         
          'word_freq_parts',        
          'word_freq_pm',           
          'word_freq_direct',       
          'word_freq_cs',           
          'word_freq_meeting',      
          'word_freq_original',     
          'word_freq_project',      
          'word_freq_re',           
          'word_freq_edu',          
          'word_freq_table',        
          'word_freq_conference',   
          'char_freq_;',            
          'char_freq_(',            
          'char_freq_[',            
          'char_freq_!',            
          'char_freq_$',            
          'char_freq_#',            
          'capital_run_length_average', 
          'capital_run_length_longest', 
          'capital_run_length_total',
          'spam']

In [4]:
# Load data
email_data = pd.read_csv('spambase.data', names=columns_data)

In [5]:
# Preview data
email_data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.64,0.00,0.00,0.00,0.32,0.00,1.29,1.93,0.00,0.96,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.0,0.06,0.06,0.0,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [6]:
email_data.sample(5)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
364,0.00,0.00,0.00,0.0,7.69,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.588,0.00,0.000,0.000,0.000,1.000,1,6,1
584,1.18,0.39,0.59,0.0,0.00,0.98,0.19,0.19,1.38,0.39,0.0,0.98,0.00,0.19,0.0,0.98,0.0,0.0,2.56,0.39,1.38,0.0,0.0,1.38,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.232,0.00,0.749,0.310,0.025,6.652,76,632,1
3156,0.00,0.00,0.56,0.0,0.08,0.16,0.00,0.00,0.00,0.16,0.0,0.00,0.00,0.24,0.0,0.00,0.0,0.0,0.24,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.08,0.08,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.54,0.164,0.505,0.00,0.010,0.021,0.000,2.729,55,1122,0
949,0.00,0.00,0.00,0.0,0.20,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.2,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.204,0.00,0.034,0.000,0.000,2.588,15,277,1
3374,0.11,0.00,0.11,0.0,0.34,0.22,0.00,0.00,1.02,0.00,0.0,0.45,0.11,0.00,0.0,0.00,0.0,0.0,0.45,0.00,0.22,0.0,0.0,0.00,0.68,0.79,0.11,0.0,0.0,0.0,0.0,0.0,0.34,0.0,0.00,0.11,0.22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.096,0.192,0.08,0.000,0.032,0.000,2.829,32,549,0


In [7]:
# Shape of data
email_data.shape

(4601, 58)

In [8]:
# Information about the data
email_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   word_freq_make              4601 non-null   float64
 1   word_freq_address           4601 non-null   float64
 2   word_freq_all               4601 non-null   float64
 3   word_freq_3d                4601 non-null   float64
 4   word_freq_our               4601 non-null   float64
 5   word_freq_over              4601 non-null   float64
 6   word_freq_remove            4601 non-null   float64
 7   word_freq_internet          4601 non-null   float64
 8   word_freq_order             4601 non-null   float64
 9   word_freq_mail              4601 non-null   float64
 10  word_freq_receive           4601 non-null   float64
 11  word_freq_will              4601 non-null   float64
 12  word_freq_people            4601 non-null   float64
 13  word_freq_report            4601 

In [9]:
# Check missing values
null_checker = email_data.isnull().sum().sort_values(ascending=False)
null_checker[null_checker > 0]

Series([], dtype: int64)

In [10]:
# Check for duplicates
email_data.duplicated().sum()

391

In [11]:
# Drop duplicates
email_data.drop_duplicates(inplace=True)

In [14]:
# Before split data check proportion of target variable
email_data.spam.value_counts(normalize=True)*100

0    60.118765
1    39.881235
Name: spam, dtype: float64

In [17]:
# Spam variable is quite imbalanced thus apply SMOTE technique to handle imbalance

# Get X and Y
X = email_data.iloc[:, :-1]
y = email_data.spam

# Apply smote to x and y
sm = SMOTE(sampling_strategy='auto', k_neighbors=1, random_state=42)
X_res, y_res = sm.fit_resample(X, y)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X_res, y_res, test_size=.3, random_state=23)

In [ ]:
# Scale data


In [ ]:
# Perform data reduction to reduce number of features
pca = PCA(n_components=57)
pca.fit(email_data.drop(['spam'], axis=1))

env = pca.explained_variance_ratio_
components = len(env) + 1
plt.plot(__builtin__.range(1, components), env.cumsum(), marker='-x-')
plt.xlable('Number ')